In [5]:
import netCDF4 as nc
from netCDF4 import Dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets.embed import embed_minimal_html
import ipywidgets as widgets
import os
os.environ["PROJ_LIB"] = "C:\\Utilities\\Python\\Anaconda\\Library\\share"; #fixr
from mpl_toolkits.basemap import Basemap
import scipy

# Downloading Data

In [4]:
years = np.arange(1980,2016)
years

array([1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990,
       1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001,
       2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,
       2013, 2014, 2015])

In [ ]:
#this downloads data from GODAS website if you do not already have data downloaded (it's under data)
for year in years:
    url = 'https://downloads.psl.noaa.gov/Datasets/godas/pottmp.'+str(year)+'.nc'
    r = requests.get(url, allow_redirects = True)
    open('pottmp.'+str(year)+'.nc', 'wb').write(r.content)

# Compiling and Reformatting Datasets

In [ ]:
file = 'data/air.mon.meanv3.nc'
air_data = nc.Dataset(file)

In [ ]:
air_lats = air_data.variables['lat'][:]
air_lons = air_data.variables['lon'][:]
time = air_data.variables['time'][:]
air = air_data.variables['air'][(1980-1836)*12::12] #only extract Jan 1980-Jan 2015
levels = air_data.variables['level'][:]
xxair, yyair = np.meshgrid(air_lons, air_lats)


In [ ]:
water_data = []
for year in years:
    file = 'data/pottmp.'+str(year)+'.nc'
    water_data.append(nc.Dataset(file))

In [ ]:
water_lons = np.array(water_data[0].variables['lon'][:])
water_lats = np.array(water_data[0].variables['lat'][:])
depths =WaterData[0].variables['level'][:]
xxwater, yywater = np.meshgrid(waterlons, waterlats)

In [ ]:
jan_temps = []
for i in range(0, len(water_data)):
    annual_air = np.array(air[i].flatten()).tolist()
    annual_water = water_data[i].variables['pottmp'][0] #Jan = 0, Feb = 1... Dec = 11
    annual_water = np.array(annual_water[0].flatten()).tolist()  #only first water layer
    jan_temps.append(annual_air + annual_water)
jan_temps = np.array(jan_temps).T
    

In [ ]:
#Replaces with Nan because of how GODAS data is formatted
for i in range(air[0].size, jan_temps.shape[0]):
    for j in range(0, jan_temps.shape[1]):
        if (jan_temps[i,j] < 0):
            jan_temps[i,j] = np.nan

# Preweighted Computations

In [ ]:
import warnings
with warnings.catch_warnings():
    warnings.simplefilter('ignore', category = RuntimeWarning) #deals with nans and dividing by 0 possibilities
    clim = np.nanmean(jan_temps, axis =1)
    sdev = np.nanstd(jan_temps, axis =1)
clim = np.reshape(clim, (clim.size,1))
sdev = np.reshape(sdev, (sdev.size,1))
    

In [ ]:
#standardized anomalies
np.seterr(divide='ignore', invalid='ignore') #to deal with divide by 0 errors and NaNs
stnd_anom = (jan_temps - clim)/ sdev


# Weighted Computations

$$ \sqrt{c_p \rho \Delta d_{ij}  \cos{\phi}\Delta \theta \Delta \phi }$$

In [ ]:
lats_air = np.array(yyair).flatten().tolist()
lats_water = np.array(yywater).flatten().tolist()

In [ ]:
lats_weighed = []
for i in range(levels.size):
    lats_weighed += lats_air
lats_weighed += lats_water
lats_weighed = np.array(lats_weighed)

In [ ]:
weighted_area = np.sqrt(np.cos(lats_weighed *np.pi/180))
weighted_area = np.reshape(weighted_area, (weighted_area,1))


### Airheights (approximated with hypsometric equations and assumed $P_0 = 1013.25$ mbar and $h_0 = 0$

$$ P_n = P_{n-1} e^{\frac{-gM(h_n-h_{n-1})}{RT}} $$

In [ ]:
air_clim =clim[0:air[0].size]

In [ ]:
#Constants
p = 101325e-2 #assumed p0
r = 8.31432 #gas constant
m = 0.0289644 
g = 9.8 #gravitational constant

In [ ]:
air_heights = []
for i in range(0, 360*181):
    air_heights.append(r*air_clim[i]/(-g*m)*np.log(levels[int(i/(181*360))]/p))
for i in range(181*360, airclim.size):
    air_heights.append(r*air_clim[i]/(-g*m)*
                       np.log(levels[int(i/(181*360))]/levels[int(i/(181*360))-1]) +air_heights[i-(181*360)])
air_heights = np.array(air_heights)

### Thickness $$ \Delta d_{ij} $$

In [ ]:
water_thickness = [5]
thickness = np.empty([stnd_anom.shape[0], 1])
thickness[0:181*360] = air_heights[0:181*360]
for i in range(181*360, air[0].size):
    thickness[i] = air_heights[i] - air_heights[i-181*360]
for j in range(air[0].size, stnd_anom.shape[0]):
    thickness[j] = water_thickness[int((j-air[0].size)/(360*418))]

### Gridbox size
$$ \Delta \theta \Delta \phi$$

In [ ]:
airgrid = 1
watergrid = 1/3

In [ ]:
gridSize = np.empty([stnd_anom.shape[0], 1])
for i in range(0, air[0].size):
    gridSize[i] = airgrid
for j in range(air[0].size, gridSize.size):
    gridSize[j] = watergrid


### Density $$\rho$$

In [ ]:
water_density = 1000
density = np.empty([stnd_anom.shape[0], 1])

Air density calculated using the NASA's Earth Atmosphere Model with given pressure and temperature.
https://www.grc.nasa.gov/www/k-12/airplane/atmosmet.html

In [ ]:
for i in range(0,air[0].size):
    density[i] =  levels[int(i/(181*360))]/(2.869*clim[i])
for j in range(air[0].size,density.size):
    density[j] = water_density

### Heat Capacity $$c_p$$

In [ ]:
cp_air = 1.005
cp_water = 4.812

In [ ]:
heatCap = np.empty([weightedAnom.shape[0], 1])
for i in range(0, air[0].size):
    heatCap[i] = cp_air
for j in range(air[0].size, gridSize.size):
    heatCap[j] = cp_water

## Putting all the different weights together

In [ ]:
vweighted_anom = stnd_anom * weighted_area * np.sqrt(thickness) * np.sqrt(density) * np.sqrt(heatCap) * np.sqrt(gridSize)

# EOF computation (Geometric and Physical)

### Eigenvalues and Eigenvectors

In [ ]:
df = pd.DataFrame(data = vweighted_anom)
dropna = df.dropna()
No_Nan_Anom = dropna.to_numpy()


In [ ]:
Sigma = np.matmul(No_Nan_Anom.T, No_Nan_Anom)
eigenvalues, eigenvectors = scipy.linalg.eig(Sigma)
eigenvectors = eigenvectors.T

In [ ]:
#sort the eigenvalues in descending order
index = np.argsort(eigenvalues)[::-1]
eigvals = engenvalues[index]
eigvecs = eigenvectors[index]


In [ ]:
num_eval = np.arange(eigvales.shape[0])+1
cumulative_eval = np.cumsum(eigvals)

### Scree Plot

In [ ]:
plt.figure(figsize=(30., 14.))
fig, ax = plt.subplots()

p1, = plt.plot(num_eval,(eigvals/cumulative_eval[-1])*100, 'b', marker = 'o',label = 'Percentage Variance')
ax.set_ylabel("Percentage Variance")
ax.yaxis.label.set_color('blue')
ax.tick_params('y', colors='b')

ax2 = ax.twinx()
p2, = plt.plot(num_eval,(cumulative_eval/cumulative_eval[-1])*100,'r', marker = 'x',label = 'Cumulative Percentage Variance')
ax2.tick_params('y', colors='r')
ax2.set_ylabel("Cumulative Percentage Variance")
ax2.yaxis.label.set_color('red')

plt.legend(handles=[p1,p2],loc='center right')

plt.show()

### EOFs

In [ ]:
EOFS = []

for j in range(0,vweightedanom.shape[1]):
        EOFS.append(np.matmul(vweighted_anom, eigvecs[j])/np.linalg.norm(np.matmul(No_Nan_Anom, eigvecs[j])))
EOF1 = np.array(EOFS).T

In [ ]:
#Physical EOFs
np.seterr(divide='ignore', invalid='ignore')
PhysicalEOFs = EOF1/(weightedA * np.sqrt(thickness) * np.sqrt(density) * np.sqrt(heatCap) * np.sqrt(gridSize))